In [89]:
# TODOs:
    # load FRED-datasets-codes.csv, extract all codes
    # fetch all data by the codes, export to csv
    # export all time series data into HBase
    # implement in spark-ts and Hbase
    # dealing with spark-ts in Scala (optional)
    # https://blog.cloudera.com/blog/2015/12/spark-ts-a-new-library-for-analyzing-time-series-data-with-apache-spark/

In [75]:
import os
import time
import random
import quandl
import pandas as pd

# Fetch FRED codes and export to CSV

In [32]:
file_path = "/Users/sundeepblue/Desktop/FRED-datasets-codes.csv"
save_path = "/Users/sundeepblue/Desktop/fred_codes/"
all_fred_codes = pd.read_csv(file_path, sep=',', header=None, names=["code_name", "description"])

## Create util functions

In [77]:
def export_one_fred_code_to_csv(code_name, save_path):
    """
    code_name: the FRED code name, eg: 'FRED/DOTSRG3Q086SBEA'
    """
    parsed_code = "_".join(code_name.split("/"))
    file_name = "{}.csv".format(parsed_code)
    full_path = os.path.join(save_path, file_name)
    
    # call Quandl API to get time series
    try:  
        series = quandl.get(code_name)
    except Exception as e:
        print e
        series = None
    
    succeed = False
    # export series to csv file
    if series is not None:
        series.to_csv(full_path)
        succeed = True
        #print "\tExport to csv succeed!"
    else:
        print "\tFailed to fetch code {}".format(code_name)
    return succeed

In [83]:
def export_all_fred_codes(all_fred_codes, save_path):
    code_names = all_fred_codes.code_name
    failed_codes = []
    for i, c in enumerate(code_names):
        if (i+1) % 50 == 0:
            print i+1, c
        status = export_one_fred_code_to_csv(c, save_path)
        if status == False:
            failed_codes.append(c)
        time.sleep(random.uniform(0, 0.25))
    return failed_codes

In [84]:
quandl.ApiConfig.api_key = 'vzT_M2k5uyw3mz1z3ynR'

## Perform actual fetching (took 4 hours to fetch 40K series)

In [85]:
# Note: It took nearly 4 hours to fetch around 40000 time series (~250Mb) via quandl api on my macbook. 
# But there are in total 194824 FRED series. I manually interrupt the fetching because it seemed unnecessary
# for me to download all of them

export_all_fred_codes(all_fred_codes, save_path)

50 FRED/SMU50000006056000001
100 FRED/PCPI02261
150 FRED/PCPI18081
200 FRED/MIAM112LFN
250 FRED/IPG315A6NQ
300 FRED/CASHAS5LFN
350 FRED/PCPI29045
400 FRED/NEKEYA3LFN
450 FRED/SMU28000006561000001
500 FRED/IPB53241SQ
550 FRED/SMU32000001021220001SA
600 FRED/SMU31365407072251101SA
650 FRED/SMU34000000600000003SA
700 FRED/ARFUURN
750 FRED/INMIAM3LFN
800 FRED/SMU27000004200000007SA
850 FRED/JANE555LFN
900 FRED/PCPI47131
950 FRED/SMU27000005051700001SA
1000 FRED/SMU28000000600000003SA
1050 FRED/SMU13000004000000011SA
1100 FRED/NMMCKI5URN
1150 FRED/NYDUTC7LFN
1200 FRED/DOTH001URN
1250 FRED/ILWARR7LFN
1300 FRED/SMU29178600800000001
1350 FRED/SMU29000003200000001
1400 FRED/ILJSURN
1450 FRED/KSCOWL5URN
1500 FRED/SMU11478944322000001SA
1550 FRED/MDCARO1LFN
1600 FRED/MNHENN3URN
1650 FRED/MIHOUG1LFN
1700 FRED/SMU27334604100000008SA
1750 FRED/OKCUST9URN
1800 FRED/SCALLE5LFN
1850 FRED/CUUSA320SAT1
1900 FRED/RSERVMIM683SFRBCHI
1950 FRED/CONSTINM683SFRBCHI
2000 FRED/SMU15000001500000007
2050 FRED/NCNE

KeyboardInterrupt: 

# Fetch S&P index and export to CSV

In [88]:
# let us now separately fetch the INDEX_GSPC-S-P-500-Index and export to csv
code = "YAHOO/INDEX_GSPC"
save_path = "/Users/sundeepblue/Desktop/gspc_code/"
export_one_fred_code_to_csv(code, save_path)

True

In [ ]:
# Now we have necessary data to use in HBase